In [1]:
import numpy as np
import math


In [26]:
class Lift(object):

    def __init__(self, terminal_vel, N_terminal, max_floors) -> None:
        self.terminal_vel = terminal_vel
        self.N_terminal = N_terminal
        self.max_floors = max_floors
        # self.t_stop = t_stop
        # pass

    location = 0
    direction = None
    num_drop_offs = 0
    num_stops = 0
    last_journey_distance = 0
    distance_travelled = 0
    total_time = 0
    time_operating = 0
    requests = {} #empty initially
    requests_to = []

    def info(self):
        print(f"Located at floor {self.location}.")
        print(f"Terminal velocity is {self.terminal_vel} floors per unit time.")
        print(f"Reaches terminal velocity after {self.N_terminal} floors.")
        print(f"Maximum floor is {self.max_floors}.")

    def request(self, floor, req_direction):
        self.requests.update({floor:req_direction})
    
    def request_to(self, floor):
        self.requests_to.append(floor)
    
    def move_to(self, floor):
        self.last_journey_distance = abs(floor-self.location)
        self.distance_travelled += abs(floor-self.location)
        self.location = floor
        self.num_stops += 1
        
    def add_time_move(self, floors_moved):
        if (floors_moved < self.N_terminal):
            moving_time = np.sqrt(4*(floors_moved*self.N_terminal)/(self.terminal_vel**2))
        else:
            moving_time = (floors_moved+self.N_terminal)/self.terminal_vel

        self.total_time += moving_time
        self.time_operating += moving_time

    # def add_time_stop(self): #So far we are ignoring stopping time.
    #     self.total_time += self.t_stop





In [27]:
test_lift = Lift(terminal_vel=2, N_terminal=3, max_floors=100)

test_lift.info()

Located at floor 0.
Terminal velocity is 2 floors per unit time.
Reaches terminal velocity after 3 floors.
Maximum floor is 100.


In [31]:
test_lift.move_to(10)
test_lift.add_time_move(1)
# test_lift.add_time_stop()

print(test_lift.location)
print(test_lift.distance_travelled)
print(test_lift.last_journey_distance)
print(test_lift.total_time)


10
50
20
6.928203230275509


In [5]:
# print(test_lift.requests)
# test_lift.request(7, "down")

# print(test_lift.requests)

{}
{7: 'down'}


## Assumptions

Assume constant acceleration until reaching terminal velocity. This termianl velocity is reached after a set number of floors $N_\mathrm{terminal}$. <br>
Assume each stop off takes the same amount of time, $t_\mathrm{stop}$. We are ignoring this to start with.

In [35]:
def Produce_request(lift):
    # Half the time it's someone on the ground floor wanting to go up
    rng = np.random.default_rng() #No seed, pulled from OS!
    number = rng.random() 

    floor_num = np.random.randint(0, lift.max_floors)
    
    if number < 0.5: #Going up
        # Request_from = {0, "up"}
        Request_from = 0
        Request_direction = "up"
        Request_to = floor_num
    else: #Going down
        # Request = {floor_num, "down"}
        Request_from = floor_num
        Request_direction = "down"
        Request_to = 0

    lift.request(Request_from, Request_direction)
    lift.request_to(Request_to)

    

In [37]:
lift_1 = Lift(terminal_vel=2, N_terminal=3, max_floors=100)
lift_2 = Lift(terminal_vel=2, N_terminal=3, max_floors=20)

lift_1.info()
lift_2.info()

print("lift 1:")
print(lift_1.requests)
print(lift_1.requests_to)

Produce_request(lift_1)

print(lift_1.requests)
print(lift_1.requests_to)

print()
print("lift 2:")
print(lift_2.requests)
print(lift_2.requests_to)

Produce_request(lift_2)

print(lift_2.requests)
print(lift_2.requests_to)

Located at floor 0.
Terminal velocity is 2 floors per unit time.
Reaches terminal velocity after 3 floors.
Maximum floor is 100.
Located at floor 0.
Terminal velocity is 2 floors per unit time.
Reaches terminal velocity after 3 floors.
Maximum floor is 20.
lift 1:
{0: 'down', 25: 'down', 14: 'down', 12: 'down'}
[0, 0, 0, 0]
{0: 'up', 25: 'down', 14: 'down', 12: 'down'}
[0, 0, 0, 0, 31]

lift 2:
{0: 'up', 25: 'down', 14: 'down', 12: 'down'}
[0, 0, 0, 0, 31]
{0: 'up', 25: 'down', 14: 'down', 12: 'down'}
[0, 0, 0, 0, 31, 0]


## Defining algorithms

In [ ]:
def Algo_linear_order(lift):
    """Completes one request at a time, in the order they arrive. No direction information used."""
    for request in lift.requests:
        lift.move_to(request)
        lift.add_time_move(Lift.last_journey_distance)

    return lift